In [1]:
import pandas as pd
import numpy as np

CREATING FINAL DATASET FROM THE FILTERED COLUMNS DATASET

In [2]:
filtered_file_path='filtered_dataset.csv.gz'

In [3]:
chunk_size = 200000  # Adjust based on memory availability
filtered_chunks = []

for chunk in pd.read_csv(filtered_file_path, chunksize=chunk_size, on_bad_lines='skip', encoding='utf-8',compression="gzip", low_memory=False):
    filtered_chunks.append(chunk)

# Concatenate all chunks into a single DataFrame
filtered_df = pd.concat(filtered_chunks, ignore_index=True)
del filtered_chunks

In [4]:
print(filtered_df.shape)

(3628367, 28)


In [5]:
beverage_keywords = [
    "Beverages and beverages preparations", "Beverages", "sodas", "drinks", "juices", "teas",
    "artificially sweetened beverages", "basil seeds drinks", "carbonated drinks", "coffee drinks", "dairy drinks",
    "dehydrated beverages", "energy drinks", "fermented drinks", "flavoured drinks", "herbal tea beverages",
    "hot beverages", "milsubstitutes", "mixed drinks", "non-alcoholic beverages", "still soft drinks",
    "plant-based beverages", "sweetened beverages", "tea-based beverages", "unsweetened beverages",
    "waters", "fr:Citronnades", "Boissons"
]

# Common global drink-related terms for substring matching in main_category
beverage_substrings = [
    "water", "drink","drinks" "juice", "soda", "teas", "coffee", "nectar", "carbonated", "milkshake", 
    "smoothie", "lemonade", "coconut-waters", "non-alcoholic", "infusion", "isotonic",
    "cold-brew", "plant-based-drinks", "kombucha", "protein-shake","tonic", 
    "ginger-beers", "electrolyte-drinks", "beverages", "ciders" 
]

# Convert to lowercase for case-insensitive matching
beverage_keywords_lower = {kw.lower() for kw in beverage_keywords}
beverage_substrings_lower = {kw.lower() for kw in beverage_substrings}

def check_categories(categories_str):
    """Check if any non-alcoholic beverage keyword exists in the 'categories' column."""
    if pd.isna(categories_str):
        return False
    category_list = [c.strip().lower() for c in categories_str.split(",")]
    return any(cat in beverage_keywords_lower for cat in category_list)

def check_main_category(main_category_str):
    """Check if 'main_category' contains any beverage-related substring."""
    if pd.isna(main_category_str):
        return False
    main_category = main_category_str.split(":", 1)[-1].strip().lower()
    parts = main_category.replace("-", " ").split()
    return any(any(sub in part for sub in beverage_substrings_lower) for part in parts)

# Apply conditions to each column
categories_condition = filtered_df['categories'].apply(check_categories)
main_category_condition = filtered_df['main_category'].apply(check_main_category)

# Combine conditions using logical OR
mask = categories_condition | main_category_condition

In [6]:
filtered_df2 = filtered_df[mask]
print(filtered_df2.shape)
print("---Before filter---")
print(filtered_df2['fruits-vegetables-nuts-estimate-from-ingredients_100g'].isnull().sum())
filtered_df2.loc[
    filtered_df2['fruits-vegetables-nuts-estimate-from-ingredients_100g'].isna() &  # If 3rd column is NaN
    (filtered_df2[['fruits-vegetables-nuts_100g', 'fruits-vegetables-nuts-estimate_100g']].notna().sum(axis=1) >= 1),  # At least one of the other two has a value
    'fruits-vegetables-nuts-estimate-from-ingredients_100g'
] = filtered_df2[['fruits-vegetables-nuts_100g', 'fruits-vegetables-nuts-estimate_100g']].mean(axis=1)

print("---After filter---")
# Check missing values after filling
print(filtered_df2['fruits-vegetables-nuts-estimate-from-ingredients_100g'].isnull().sum())

(173379, 28)
---Before filter---
79635
---After filter---
79312


In [7]:
filtered_df3 = filtered_df2.drop(columns=['countries','fat_100g','allergens','energy-kcal_100g','energy-kj_100g','trans-fat_100g','added-sugars_100g','sodium_100g','chloride_100g','fruits-vegetables-nuts_100g', 'fruits-vegetables-nuts-estimate_100g', 'nutrition-score-fr_100g',])
filtered_df3.shape

(173379, 16)

In [8]:
# List of columns to check for non-null values
columns_to_check = [
    'energy_100g', 'saturated-fat_100g', 'sugars_100g', 
    'fiber_100g', 'proteins_100g', 'salt_100g', 'additives_tags',
    'fruits-vegetables-nuts-estimate-from-ingredients_100g'
]

# Count the number of non-null values in the specified columns
non_null_count = filtered_df3[columns_to_check].notnull().sum(axis=1)

# Keep only rows where at least 4 of the columns have values
filtered_df4 = filtered_df3[non_null_count >= 5]

filtered_df4.loc[:, columns_to_check] = filtered_df4.loc[:, columns_to_check].fillna(0)

# Verify the changes
filtered_df4.shape

(109114, 16)

In [9]:
filtered_df5 = filtered_df4[~filtered_df4['nutriscore_grade'].isin(['not-applicable'])]
filtered_df5.shape

(106276, 16)

In [10]:
print(filtered_df5['nutriscore_grade'].unique())

['unknown' 'c' 'e' 'b' 'd' 'a']


In [11]:
filtered_df5.columns = filtered_df5.columns.str.replace('-', '_')
filtered_df5 = filtered_df5.rename(columns={
    'fruits_vegetables_nuts_estimate_from_ingredients_100g': 'fruits_veg_nuts_100g'
})
filtered_df5.columns

Index(['product_name', 'categories', 'main_category', 'quantity',
       'ingredients_tags', 'additives_tags', 'energy_100g',
       'saturated_fat_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g',
       'salt_100g', 'alcohol_100g', 'fruits_veg_nuts_100g', 'nutriscore_score',
       'nutriscore_grade'],
      dtype='object')

In [12]:
import pandas as pd

# Create a condition where all the relevant columns are 0
condition = (filtered_df5[['energy_100g', 'saturated_fat_100g', 'sugars_100g', 'fiber_100g', 
                           'proteins_100g', 'salt_100g', 'fruits_veg_nuts_100g']].eq(0)).all(axis=1)

# Filter out rows where all the relevant columns are 0, and nutriscore_grade is between 'c' and 'e'
new_filtered_df = filtered_df5[~(condition & filtered_df5['nutriscore_grade'].isin(['c', 'd', 'e']))]

# Convert columns to numeric, forcing errors to NaN (this will handle any invalid strings or power notation)
cols_to_check = ['energy_100g', 'saturated_fat_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'fruits_veg_nuts_100g']

new_filtered_df[cols_to_check] = new_filtered_df[cols_to_check].apply(pd.to_numeric, errors='coerce')

# Define the conditions where any one column exceeds the threshold
condition1 = (
    (new_filtered_df['energy_100g'] > 500) | (new_filtered_df['energy_100g'] < 0) |
    (new_filtered_df['saturated_fat_100g'] > 20) | (new_filtered_df['saturated_fat_100g'] < 0) |
    (new_filtered_df['sugars_100g'] > 25) | (new_filtered_df['sugars_100g'] < 0) |
    (new_filtered_df['proteins_100g'] > 10) | (new_filtered_df['proteins_100g'] < 0) |
    (new_filtered_df['salt_100g'] > 15) | (new_filtered_df['salt_100g'] < 0) |
    (new_filtered_df['fiber_100g'] > 10) | (new_filtered_df['fiber_100g'] < 0) |
    (new_filtered_df['fruits_veg_nuts_100g'] > 100) | (new_filtered_df['fruits_veg_nuts_100g'] < 0)
)

# Remove rows where any condition is matched
new_filtered_df1 = new_filtered_df[~condition1]

# Display the filtered dataframe
new_filtered_df1.shape

C:\Users\apran\AppData\Local\Temp\ipykernel_21156\1644439046.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_filtered_df[cols_to_check] = new_filtered_df[cols_to_check].apply(pd.to_numeric, errors='coerce')


(90468, 16)

In [13]:
grade_a_products = new_filtered_df1[new_filtered_df1['nutriscore_grade'] == 'a']
other_products = new_filtered_df1[new_filtered_df1['nutriscore_grade'] != 'a']

other_products['nutriscore_grade'].unique()

array(['unknown', 'd', 'c', 'e', 'b'], dtype=object)

In [17]:
import pandas as pd
import numpy as np

# =============================================
# Unfavorable Components (N Points Calculation)
# =============================================

def calculate_energy_points(energy):
    thresholds = [30, 90, 150, 210, 240, 270, 300, 330, 360, 390]
    for i, thresh in enumerate(thresholds):
        if energy <= thresh:
            return i
    return 10  # >390

def calculate_saturated_fat_points(sat_fat):
    thresholds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    for i, thresh in enumerate(thresholds):
        if sat_fat <= thresh:
            return i
    return 10  # >10

def calculate_salt_points(salt):
    thresholds = [0.2, 0.4, 0.6, 0.8, 1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4, 2.6, 2.8, 3, 3.2, 3.4, 3.6, 3.8, 4]
    for i, thresh in enumerate(thresholds):
        if salt <= thresh:
            return i
    return 20  # >4

def calculate_sugar_points(sugar):
    thresholds = [0.5, 2, 3.5, 5, 6, 7, 8, 9, 10, 11]
    for i, thresh in enumerate(thresholds):
        if sugar <= thresh:
            return i
    return 10  # >11

def calculate_sweetener_points(additives_tags):
    sweeteners = {'e420', 'e421', 'e953', 'e956', 'e964', 'e965', 'e966', 'e967', 'e968'}
    if pd.isna(additives_tags):
        return 0
    # Split comma-separated additives and extract E-numbers
    additives = [additive.split(':')[-1].lower() for additive in str(additives_tags).split(',')]
    return 4 if any(additive in sweeteners for additive in additives) else 0

# ===========================================
# Favorable Components (P Points Calculation)
# ===========================================

def calculate_protein_points(protein):
    thresholds = [1.2, 1.5, 1.8, 2.1, 2.4, 2.7, 3.0]
    for i, thresh in enumerate(thresholds):
        if protein <= thresh:
            return i
    return 7  # >3.0

def calculate_fiber_points(fiber):
    thresholds = [3.0, 4.1, 5.2, 6.3, 7.4]
    for i, thresh in enumerate(thresholds):
        if fiber <= thresh:
            return i
    return 5  # >7.4

def calculate_fvp_points(fvp_percent):
    if fvp_percent <= 40:
        return 0
    elif fvp_percent <= 60:
        return 2
    elif fvp_percent <= 80:
        return 4
    else:
        return 6

# ========================
# Main Calculation
# ========================

def calculate_nutri_score(row):
    try:
        # Calculate N Points (Unfavorable)
        n_energy = calculate_energy_points(row['energy_100g'])
        n_sat_fat = calculate_saturated_fat_points(row['saturated_fat_100g'])
        n_salt = calculate_salt_points(row['salt_100g'])
        n_sugar = calculate_sugar_points(row['sugars_100g'])
        n_sweetener = calculate_sweetener_points(row.get('additives_tags', np.nan))
        
        total_n = n_energy + n_sat_fat + n_salt + n_sugar + n_sweetener
        
        # Calculate P Points (Favorable)
        p_protein = calculate_protein_points(row['proteins_100g'])
        p_fiber = calculate_fiber_points(row['fiber_100g'])
        p_fvp = calculate_fvp_points(row['fruits_veg_nuts_100g'])
        
        total_p = p_protein + p_fiber + p_fvp
        
        # Final Nutri-Score
        return total_n - total_p
        
    except (TypeError, KeyError):
        return np.nan

def score_to_grade(score):
    if pd.isna(score):
        return np.nan
    if score <= 2:
        return 'b'
    elif score <= 6:
        return 'c'
    elif score <= 9:
        return 'd'
    else:
        return 'e'


# Step 1: Separate products with grade 'A' and others
grade_a_products = new_filtered_df1[new_filtered_df1['nutriscore_grade'] == 'a']
other_products = new_filtered_df1[new_filtered_df1['nutriscore_grade'] != 'a']

# Step 2: Convert relevant columns to numeric for other products
nutrition_cols = ['additives_tags', 'energy_100g',
       'saturated_fat_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g',
       'salt_100g','fruits_veg_nuts_100g']
for col in nutrition_cols:
    other_products[col] = pd.to_numeric(other_products[col], errors='coerce')

# Step 3: Apply the new calculation logic to other products
other_products['nutriscore_score'] = other_products.apply(calculate_nutri_score, axis=1)
other_products['nutriscore_grade'] = other_products['nutriscore_score'].apply(score_to_grade)

# Step 4: Combine the 'A' grade products back with the recalculated products
final_df = pd.concat([other_products], axis=0)

final_df['nutriscore_grade'].unique()

C:\Users\apran\AppData\Local\Temp\ipykernel_21156\3313109372.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_products[col] = pd.to_numeric(other_products[col], errors='coerce')
C:\Users\apran\AppData\Local\Temp\ipykernel_21156\3313109372.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_products['nutriscore_score'] = other_products.apply(calculate_nutri_score, axis=1)
C:\Users\apran\AppData\Local\Temp\ipykernel_21156\3313109372.py:126: SettingWithCopyWarning: 
A value is trying to be s

array(['b', 'e', 'c', 'd'], dtype=object)

In [18]:
filtered_file = "final_dataset.csv"

num_features = [
    'energy_100g', 'saturated_fat_100g', 'sugars_100g',
    'fiber_100g', 'proteins_100g', 'salt_100g',
    'fruits_veg_nuts_100g',
]
label = ['nutriscore_grade']

# Split data
newfinalData = final_df[num_features + label]
newfinalData.to_csv(filtered_file, index=False,encoding='utf-8')
newfinalData.shape

(87588, 8)

In [233]:
newfinalData.describe()

,energy_100g,saturated_fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,fruits_veg_nuts_100g
count,91203.000000,91203.000000,91203.000000,91203.000000,91203.000000,91203.000000,91203.000000
mean,131.362209,0.152819,5.763161,0.141003,0.554967,0.098203,14.353786
std,100.620656,0.585540,4.822336,0.481483,1.195732,0.649753,29.313120
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,21.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000
50%,140.000000,0.000000,5.420000,0.000000,0.000000,0.010000,0.000000
75%,201.000000,0.100000,10.000000,0.000000,0.500000,0.052500,11.000000
max,499.000000,20.000000,25.000000,10.000000,10.000000,15.000000,100.000000


In [ ]:
import pandas as pd

# Create a condition where all the relevant columns are 0
condition = (filtered_df4[['energy_100g', 'saturated_fat_100g', 'sugars_100g', 'fiber_100g', 
                           'proteins_100g', 'salt_100g', 'fruits_veg_nuts_100g']].eq(0)).all(axis=1)

# Filter out rows where all the relevant columns are 0, and nutriscore_grade is between 'b' and 'e'
new_filtered_df = filtered_df4[~(condition & filtered_df4['nutriscore_grade'].isin(['b', 'c', 'd', 'e']))]

# Convert columns to numeric, forcing errors to NaN (this will handle any invalid strings or power notation)
cols_to_check = ['energy_100g', 'saturated_fat_100g', 'sugars_100g', 'fiber_100g', 'proteins_100g', 'salt_100g', 'fruits_veg_nuts_100g']

new_filtered_df[cols_to_check] = new_filtered_df[cols_to_check].apply(pd.to_numeric, errors='coerce')

# Define the conditions where any one column exceeds the threshold
condition1 = (
    (new_filtered_df['energy_100g'] > 500) | (new_filtered_df['energy_100g'] < 0) |
    (new_filtered_df['saturated_fat_100g'] > 20) | (new_filtered_df['saturated_fat_100g'] < 0) |
    (new_filtered_df['sugars_100g'] > 25) | (new_filtered_df['sugars_100g'] < 0) |
    (new_filtered_df['proteins_100g'] > 10) | (new_filtered_df['proteins_100g'] < 0) |
    (new_filtered_df['salt_100g'] > 15) | (new_filtered_df['salt_100g'] < 0) |
    (new_filtered_df['fiber_100g'] > 10) | (new_filtered_df['fiber_100g'] < 0) |
    (new_filtered_df['fruits_veg_nuts_100g'] > 100) | (new_filtered_df['fruits_veg_nuts_100g'] < 0)
)

# Remove rows where any condition is matched
new_filtered_df1 = new_filtered_df[~condition1]

# Display the filtered dataframe
new_filtered_df1.sample(2)

C:\Users\apran\AppData\Local\Temp\ipykernel_9380\410018323.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_filtered_df[cols_to_check] = new_filtered_df[cols_to_check].apply(pd.to_numeric, errors='coerce')


,product_name,main_category,quantity,ingredients_tags,additives_tags,energy_100g,saturated_fat_100g,sugars_100g,fiber_100g,proteins_100g,salt_100g,fruits_veg_nuts_100g,nutriscore_score,nutriscore_grade
335159,Soft drink,en:sodas,unknown,"en:contains,en:high-fructose-corn-syrup,en:add...","en:e129,en:e150c,en:e211,en:e330,en:e338",176.0,NaN,11.67,NaN,0.0,0.0175,0.0,13.0,e
3441104,Better Water Energy : Ginseng Cranberry Pomegr...,en:beverages-with-cranberry,75cl,unknown,NaN,17.0,0.0,0.80,NaN,0.0,0.0000,NaN,1.0,b


MAKING DATA FOR NLP PROCESSING

In [ ]:
import pandas as pd

# Load the dataset
file_path = "final_dataset.csv"
df = pd.read_csv(file_path)

# Define the columns to include in the text column
columns_to_include = [
    "energy_100g", "saturated_fat_100g", "sugars_100g", "fiber_100g",
    "proteins_100g", "salt_100g", "fruits_veg_nuts_100g", 
]

# Populate the 'text' column without extra decimal points
df["text"] = df[columns_to_include].apply(lambda row: ','.join(f"{col}:{int(row[col]) if row[col] % 1 == 0 else row[col]}" for col in columns_to_include), axis=1)

# Save the modified dataset
df = df.drop(columns=["energy_100g", "saturated_fat_100g", "sugars_100g", "fiber_100g", 
                      "proteins_100g", "salt_100g", "fruits_veg_nuts_100g", "sweetener_present"])
output_file = "final_dataset_updated.csv"
df.to_csv(output_file,index=False)